In [1]:
from fetch.match_results import read_joined 

df, player_mapping, inverse_player_mapping = read_joined()

/Users/siddhantjagadish/Documents/DataProjects/tennis_modeling/tennis_modeling/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2818: DtypeWarning: Columns (14,30) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [2]:
df['__surface__'].value_counts(normalize=True)

Hard      0.385820
Clay      0.326738
Indoor    0.178666
Grass     0.108776
Name: __surface__, dtype: float64

In [3]:
df.groupby(['court', 'surface']).size() / df.shape[0]

court    surface
Indoor   Carpet     0.034096
         Clay       0.003652
         Hard       0.140917
Outdoor  Clay       0.326738
         Grass      0.108776
         Hard       0.385820
dtype: float64

In [4]:
df.sort('date', inplace=True)

/Users/siddhantjagadish/Documents/DataProjects/tennis_modeling/tennis_modeling/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  """Entry point for launching an IPython kernel.


#### Build a Model for Just Clay 

In [5]:
import itertools
import numpy as np
import pandas as pd
from models.logit.base import sipko_weights, get_X_y
from ml.prior_logit import NonZeroLogit
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression


N_ATTEMPTS_PER_SURFACE = 10
SURFACES = df['__surface__'].unique().tolist()
N_SURFACES = len(SURFACES) 
TUNERS = list(itertools.product(*[
    np.linspace(0., 1., N_ATTEMPTS_PER_SURFACE + 1) for _ in range(N_SURFACES - 1)
]))
DISCOUNT = 0.8  # Taken from Sipko paper
FLAT_TIME = 1.  # Taken from Sipko paper
LMBDA = 10.  # Regularize!
PRIOR = -2.


def _get_surface_weights(train_df, surface_weight_map):
    return train_df['__surface__'].map(lambda x: surface_weight_map[x])


def try_surface_weights(train_df, sw_map, val_df, sipko_weighting):
    surface_weights = _get_surface_weights(train_df, sw_map)
    sample_weights = sipko_weighting * surface_weights
    nzl = NonZeroLogit(lmbda=LMBDA, prior=PRIOR, seed=10)
    nzl.fit(train_X, train_y, sample_weight=sample_weights)
    val_preds = nzl.predict_proba(val_X)[:, 1]
    auc = roc_auc_score(val_df['y'], val_preds)
    accuracy = (val_df['y'] == (val_preds > 0.5).astype(int)).mean()
    return auc, accuracy


def tune_weights_for_surface(surface):
    # Get data
    train_df = df[
        (df['date'] < '2016-01-01') 
    ].copy()
    train_X, train_y = get_X_y(train_df, player_mapping)
    val_df = df[
        (df['date'] >= '2016-01-01') &
        (df['date'] < '2017-01-01') &
        (df['__surface__'] == surface) &
        (~df['comment'].isin(['Retired', 'Walkover']))
    ].copy()
    val_X, val_y = get_X_y(val_df, player_mapping)
    s_weights = sipko_weights(train_df['date'].max(), train_df, DISCOUNT, flat_time=FLAT_TIME)
    
    tune_surfaces = sorted([x for x in SURFACES if x != surface])

    perfs = []
    for weights in TUNERS:
        sw_map = dict(zip(tune_surfaces, weights))
        sw_map.update({surface: 1.})
        print sw_map
        surface_weights = _get_surface_weights(train_df, sw_map)
        sample_weights = s_weights * surface_weights

        # mod = NonZeroLogit(lmbda=LMBDA, prior=PRIOR, seed=10)
        mod = LogisticRegression(C=0.4)
        mod.fit(train_X, train_y, sample_weight=sample_weights)

        val_preds = mod.predict_proba(val_X)[:, 1]
        auc = roc_auc_score(val_df['y'], val_preds)
        accuracy = (val_df['y'] == (val_preds > 0.5).astype(int)).mean()

        _record = list(weights)
        _record.extend([auc, accuracy])
        perfs.append(_record)
    return pd.DataFrame(
        perfs,
        columns=tune_surfaces + ['auc', 'accuracy']
    )

In [6]:
hard_results = tune_weights_for_surface('Hard')

{'Hard': 1.0, 'Grass': 0.0, 'Indoor': 0.0, 'Clay': 0.0}
{'Hard': 1.0, 'Grass': 0.0, 'Indoor': 0.10000000000000001, 'Clay': 0.0}
{'Hard': 1.0, 'Grass': 0.0, 'Indoor': 0.20000000000000001, 'Clay': 0.0}
{'Hard': 1.0, 'Grass': 0.0, 'Indoor': 0.30000000000000004, 'Clay': 0.0}
{'Hard': 1.0, 'Grass': 0.0, 'Indoor': 0.40000000000000002, 'Clay': 0.0}
{'Hard': 1.0, 'Grass': 0.0, 'Indoor': 0.5, 'Clay': 0.0}
{'Hard': 1.0, 'Grass': 0.0, 'Indoor': 0.60000000000000009, 'Clay': 0.0}
{'Hard': 1.0, 'Grass': 0.0, 'Indoor': 0.70000000000000007, 'Clay': 0.0}
{'Hard': 1.0, 'Grass': 0.0, 'Indoor': 0.80000000000000004, 'Clay': 0.0}
{'Hard': 1.0, 'Grass': 0.0, 'Indoor': 0.90000000000000002, 'Clay': 0.0}
{'Hard': 1.0, 'Grass': 0.0, 'Indoor': 1.0, 'Clay': 0.0}
{'Hard': 1.0, 'Grass': 0.10000000000000001, 'Indoor': 0.0, 'Clay': 0.0}
{'Hard': 1.0, 'Grass': 0.10000000000000001, 'Indoor': 0.10000000000000001, 'Clay': 0.0}
{'Hard': 1.0, 'Grass': 0.10000000000000001, 'Indoor': 0.20000000000000001, 'Clay': 0.0}
{'Hard':

{'Hard': 1.0, 'Grass': 0.90000000000000002, 'Indoor': 0.5, 'Clay': 0.0}
{'Hard': 1.0, 'Grass': 0.90000000000000002, 'Indoor': 0.60000000000000009, 'Clay': 0.0}
{'Hard': 1.0, 'Grass': 0.90000000000000002, 'Indoor': 0.70000000000000007, 'Clay': 0.0}
{'Hard': 1.0, 'Grass': 0.90000000000000002, 'Indoor': 0.80000000000000004, 'Clay': 0.0}
{'Hard': 1.0, 'Grass': 0.90000000000000002, 'Indoor': 0.90000000000000002, 'Clay': 0.0}
{'Hard': 1.0, 'Grass': 0.90000000000000002, 'Indoor': 1.0, 'Clay': 0.0}
{'Hard': 1.0, 'Grass': 1.0, 'Indoor': 0.0, 'Clay': 0.0}
{'Hard': 1.0, 'Grass': 1.0, 'Indoor': 0.10000000000000001, 'Clay': 0.0}
{'Hard': 1.0, 'Grass': 1.0, 'Indoor': 0.20000000000000001, 'Clay': 0.0}
{'Hard': 1.0, 'Grass': 1.0, 'Indoor': 0.30000000000000004, 'Clay': 0.0}
{'Hard': 1.0, 'Grass': 1.0, 'Indoor': 0.40000000000000002, 'Clay': 0.0}
{'Hard': 1.0, 'Grass': 1.0, 'Indoor': 0.5, 'Clay': 0.0}
{'Hard': 1.0, 'Grass': 1.0, 'Indoor': 0.60000000000000009, 'Clay': 0.0}
{'Hard': 1.0, 'Grass': 1.0, 'Ind

{'Hard': 1.0, 'Grass': 0.60000000000000009, 'Indoor': 0.90000000000000002, 'Clay': 0.10000000000000001}
{'Hard': 1.0, 'Grass': 0.60000000000000009, 'Indoor': 1.0, 'Clay': 0.10000000000000001}
{'Hard': 1.0, 'Grass': 0.70000000000000007, 'Indoor': 0.0, 'Clay': 0.10000000000000001}
{'Hard': 1.0, 'Grass': 0.70000000000000007, 'Indoor': 0.10000000000000001, 'Clay': 0.10000000000000001}
{'Hard': 1.0, 'Grass': 0.70000000000000007, 'Indoor': 0.20000000000000001, 'Clay': 0.10000000000000001}
{'Hard': 1.0, 'Grass': 0.70000000000000007, 'Indoor': 0.30000000000000004, 'Clay': 0.10000000000000001}
{'Hard': 1.0, 'Grass': 0.70000000000000007, 'Indoor': 0.40000000000000002, 'Clay': 0.10000000000000001}
{'Hard': 1.0, 'Grass': 0.70000000000000007, 'Indoor': 0.5, 'Clay': 0.10000000000000001}
{'Hard': 1.0, 'Grass': 0.70000000000000007, 'Indoor': 0.60000000000000009, 'Clay': 0.10000000000000001}
{'Hard': 1.0, 'Grass': 0.70000000000000007, 'Indoor': 0.70000000000000007, 'Clay': 0.10000000000000001}
{'Hard':

{'Hard': 1.0, 'Grass': 0.30000000000000004, 'Indoor': 0.90000000000000002, 'Clay': 0.20000000000000001}
{'Hard': 1.0, 'Grass': 0.30000000000000004, 'Indoor': 1.0, 'Clay': 0.20000000000000001}
{'Hard': 1.0, 'Grass': 0.40000000000000002, 'Indoor': 0.0, 'Clay': 0.20000000000000001}
{'Hard': 1.0, 'Grass': 0.40000000000000002, 'Indoor': 0.10000000000000001, 'Clay': 0.20000000000000001}
{'Hard': 1.0, 'Grass': 0.40000000000000002, 'Indoor': 0.20000000000000001, 'Clay': 0.20000000000000001}
{'Hard': 1.0, 'Grass': 0.40000000000000002, 'Indoor': 0.30000000000000004, 'Clay': 0.20000000000000001}
{'Hard': 1.0, 'Grass': 0.40000000000000002, 'Indoor': 0.40000000000000002, 'Clay': 0.20000000000000001}
{'Hard': 1.0, 'Grass': 0.40000000000000002, 'Indoor': 0.5, 'Clay': 0.20000000000000001}
{'Hard': 1.0, 'Grass': 0.40000000000000002, 'Indoor': 0.60000000000000009, 'Clay': 0.20000000000000001}
{'Hard': 1.0, 'Grass': 0.40000000000000002, 'Indoor': 0.70000000000000007, 'Clay': 0.20000000000000001}
{'Hard':

{'Hard': 1.0, 'Grass': 0.0, 'Indoor': 0.90000000000000002, 'Clay': 0.30000000000000004}
{'Hard': 1.0, 'Grass': 0.0, 'Indoor': 1.0, 'Clay': 0.30000000000000004}
{'Hard': 1.0, 'Grass': 0.10000000000000001, 'Indoor': 0.0, 'Clay': 0.30000000000000004}
{'Hard': 1.0, 'Grass': 0.10000000000000001, 'Indoor': 0.10000000000000001, 'Clay': 0.30000000000000004}
{'Hard': 1.0, 'Grass': 0.10000000000000001, 'Indoor': 0.20000000000000001, 'Clay': 0.30000000000000004}
{'Hard': 1.0, 'Grass': 0.10000000000000001, 'Indoor': 0.30000000000000004, 'Clay': 0.30000000000000004}
{'Hard': 1.0, 'Grass': 0.10000000000000001, 'Indoor': 0.40000000000000002, 'Clay': 0.30000000000000004}
{'Hard': 1.0, 'Grass': 0.10000000000000001, 'Indoor': 0.5, 'Clay': 0.30000000000000004}
{'Hard': 1.0, 'Grass': 0.10000000000000001, 'Indoor': 0.60000000000000009, 'Clay': 0.30000000000000004}
{'Hard': 1.0, 'Grass': 0.10000000000000001, 'Indoor': 0.70000000000000007, 'Clay': 0.30000000000000004}
{'Hard': 1.0, 'Grass': 0.100000000000000

...

{'Hard': 1.0, 'Grass': 0.60000000000000009, 'Indoor': 0.70000000000000007, 'Clay': 0.70000000000000007}
{'Hard': 1.0, 'Grass': 0.60000000000000009, 'Indoor': 0.80000000000000004, 'Clay': 0.70000000000000007}
{'Hard': 1.0, 'Grass': 0.60000000000000009, 'Indoor': 0.90000000000000002, 'Clay': 0.70000000000000007}
{'Hard': 1.0, 'Grass': 0.60000000000000009, 'Indoor': 1.0, 'Clay': 0.70000000000000007}
{'Hard': 1.0, 'Grass': 0.70000000000000007, 'Indoor': 0.0, 'Clay': 0.70000000000000007}
{'Hard': 1.0, 'Grass': 0.70000000000000007, 'Indoor': 0.10000000000000001, 'Clay': 0.70000000000000007}
{'Hard': 1.0, 'Grass': 0.70000000000000007, 'Indoor': 0.20000000000000001, 'Clay': 0.70000000000000007}
{'Hard': 1.0, 'Grass': 0.70000000000000007, 'Indoor': 0.30000000000000004, 'Clay': 0.70000000000000007}
{'Hard': 1.0, 'Grass': 0.70000000000000007, 'Indoor': 0.40000000000000002, 'Clay': 0.70000000000000007}
{'Hard': 1.0, 'Grass': 0.70000000000000007, 'Indoor': 0.5, 'Clay': 0.70000000000000007}
{'Hard':

{'Hard': 1.0, 'Grass': 0.30000000000000004, 'Indoor': 0.70000000000000007, 'Clay': 0.80000000000000004}
{'Hard': 1.0, 'Grass': 0.30000000000000004, 'Indoor': 0.80000000000000004, 'Clay': 0.80000000000000004}
{'Hard': 1.0, 'Grass': 0.30000000000000004, 'Indoor': 0.90000000000000002, 'Clay': 0.80000000000000004}
{'Hard': 1.0, 'Grass': 0.30000000000000004, 'Indoor': 1.0, 'Clay': 0.80000000000000004}
{'Hard': 1.0, 'Grass': 0.40000000000000002, 'Indoor': 0.0, 'Clay': 0.80000000000000004}
{'Hard': 1.0, 'Grass': 0.40000000000000002, 'Indoor': 0.10000000000000001, 'Clay': 0.80000000000000004}
{'Hard': 1.0, 'Grass': 0.40000000000000002, 'Indoor': 0.20000000000000001, 'Clay': 0.80000000000000004}
{'Hard': 1.0, 'Grass': 0.40000000000000002, 'Indoor': 0.30000000000000004, 'Clay': 0.80000000000000004}
{'Hard': 1.0, 'Grass': 0.40000000000000002, 'Indoor': 0.40000000000000002, 'Clay': 0.80000000000000004}
{'Hard': 1.0, 'Grass': 0.40000000000000002, 'Indoor': 0.5, 'Clay': 0.80000000000000004}
{'Hard':

{'Hard': 1.0, 'Grass': 0.0, 'Indoor': 0.70000000000000007, 'Clay': 0.90000000000000002}
{'Hard': 1.0, 'Grass': 0.0, 'Indoor': 0.80000000000000004, 'Clay': 0.90000000000000002}
{'Hard': 1.0, 'Grass': 0.0, 'Indoor': 0.90000000000000002, 'Clay': 0.90000000000000002}
{'Hard': 1.0, 'Grass': 0.0, 'Indoor': 1.0, 'Clay': 0.90000000000000002}
{'Hard': 1.0, 'Grass': 0.10000000000000001, 'Indoor': 0.0, 'Clay': 0.90000000000000002}
{'Hard': 1.0, 'Grass': 0.10000000000000001, 'Indoor': 0.10000000000000001, 'Clay': 0.90000000000000002}
{'Hard': 1.0, 'Grass': 0.10000000000000001, 'Indoor': 0.20000000000000001, 'Clay': 0.90000000000000002}
{'Hard': 1.0, 'Grass': 0.10000000000000001, 'Indoor': 0.30000000000000004, 'Clay': 0.90000000000000002}
{'Hard': 1.0, 'Grass': 0.10000000000000001, 'Indoor': 0.40000000000000002, 'Clay': 0.90000000000000002}
{'Hard': 1.0, 'Grass': 0.10000000000000001, 'Indoor': 0.5, 'Clay': 0.90000000000000002}
{'Hard': 1.0, 'Grass': 0.10000000000000001, 'Indoor': 0.6000000000000000

{'Hard': 1.0, 'Grass': 0.80000000000000004, 'Indoor': 0.70000000000000007, 'Clay': 0.90000000000000002}
{'Hard': 1.0, 'Grass': 0.80000000000000004, 'Indoor': 0.80000000000000004, 'Clay': 0.90000000000000002}
{'Hard': 1.0, 'Grass': 0.80000000000000004, 'Indoor': 0.90000000000000002, 'Clay': 0.90000000000000002}
{'Hard': 1.0, 'Grass': 0.80000000000000004, 'Indoor': 1.0, 'Clay': 0.90000000000000002}
{'Hard': 1.0, 'Grass': 0.90000000000000002, 'Indoor': 0.0, 'Clay': 0.90000000000000002}
{'Hard': 1.0, 'Grass': 0.90000000000000002, 'Indoor': 0.10000000000000001, 'Clay': 0.90000000000000002}
{'Hard': 1.0, 'Grass': 0.90000000000000002, 'Indoor': 0.20000000000000001, 'Clay': 0.90000000000000002}
{'Hard': 1.0, 'Grass': 0.90000000000000002, 'Indoor': 0.30000000000000004, 'Clay': 0.90000000000000002}
{'Hard': 1.0, 'Grass': 0.90000000000000002, 'Indoor': 0.40000000000000002, 'Clay': 0.90000000000000002}
{'Hard': 1.0, 'Grass': 0.90000000000000002, 'Indoor': 0.5, 'Clay': 0.90000000000000002}
{'Hard':

{'Hard': 1.0, 'Grass': 0.60000000000000009, 'Indoor': 0.80000000000000004, 'Clay': 1.0}
{'Hard': 1.0, 'Grass': 0.60000000000000009, 'Indoor': 0.90000000000000002, 'Clay': 1.0}
{'Hard': 1.0, 'Grass': 0.60000000000000009, 'Indoor': 1.0, 'Clay': 1.0}
{'Hard': 1.0, 'Grass': 0.70000000000000007, 'Indoor': 0.0, 'Clay': 1.0}
{'Hard': 1.0, 'Grass': 0.70000000000000007, 'Indoor': 0.10000000000000001, 'Clay': 1.0}
{'Hard': 1.0, 'Grass': 0.70000000000000007, 'Indoor': 0.20000000000000001, 'Clay': 1.0}
{'Hard': 1.0, 'Grass': 0.70000000000000007, 'Indoor': 0.30000000000000004, 'Clay': 1.0}
{'Hard': 1.0, 'Grass': 0.70000000000000007, 'Indoor': 0.40000000000000002, 'Clay': 1.0}
{'Hard': 1.0, 'Grass': 0.70000000000000007, 'Indoor': 0.5, 'Clay': 1.0}
{'Hard': 1.0, 'Grass': 0.70000000000000007, 'Indoor': 0.60000000000000009, 'Clay': 1.0}
{'Hard': 1.0, 'Grass': 0.70000000000000007, 'Indoor': 0.70000000000000007, 'Clay': 1.0}
{'Hard': 1.0, 'Grass': 0.70000000000000007, 'Indoor': 0.80000000000000004, 'Clay

In [8]:
hard_results.sort('auc', ascending=False).head(15)

/Users/siddhantjagadish/Documents/DataProjects/tennis_modeling/tennis_modeling/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  """Entry point for launching an IPython kernel.


,Clay,Grass,Indoor,auc,accuracy
1088,0.8,1.0,1.0,0.743780,0.665289
1209,0.9,1.0,1.0,0.743681,0.668388
1077,0.8,0.9,1.0,0.743668,0.667355
1176,0.9,0.7,1.0,0.743553,0.669421
1308,1.0,0.8,1.0,0.743536,0.671488
835,0.6,0.9,1.0,0.743536,0.670455
1198,0.9,0.9,1.0,0.743518,0.666322
1317,1.0,0.9,0.8,0.743518,0.670455
1207,0.9,1.0,0.8,0.743514,0.672521
1186,0.9,0.8,0.9,0.743514,0.674587


In [9]:
clay_results = tune_weights_for_surface('Clay')

{'Grass': 0.0, 'Indoor': 0.0, 'Clay': 1.0, 'Hard': 0.0}
{'Grass': 0.0, 'Indoor': 0.10000000000000001, 'Clay': 1.0, 'Hard': 0.0}
{'Grass': 0.0, 'Indoor': 0.20000000000000001, 'Clay': 1.0, 'Hard': 0.0}
{'Grass': 0.0, 'Indoor': 0.30000000000000004, 'Clay': 1.0, 'Hard': 0.0}
{'Grass': 0.0, 'Indoor': 0.40000000000000002, 'Clay': 1.0, 'Hard': 0.0}
{'Grass': 0.0, 'Indoor': 0.5, 'Clay': 1.0, 'Hard': 0.0}
{'Grass': 0.0, 'Indoor': 0.60000000000000009, 'Clay': 1.0, 'Hard': 0.0}
{'Grass': 0.0, 'Indoor': 0.70000000000000007, 'Clay': 1.0, 'Hard': 0.0}
{'Grass': 0.0, 'Indoor': 0.80000000000000004, 'Clay': 1.0, 'Hard': 0.0}
{'Grass': 0.0, 'Indoor': 0.90000000000000002, 'Clay': 1.0, 'Hard': 0.0}
{'Grass': 0.0, 'Indoor': 1.0, 'Clay': 1.0, 'Hard': 0.0}
{'Grass': 0.0, 'Indoor': 0.0, 'Clay': 1.0, 'Hard': 0.10000000000000001}
{'Grass': 0.0, 'Indoor': 0.10000000000000001, 'Clay': 1.0, 'Hard': 0.10000000000000001}
{'Grass': 0.0, 'Indoor': 0.20000000000000001, 'Clay': 1.0, 'Hard': 0.10000000000000001}
{'Grass'

{'Grass': 0.0, 'Indoor': 0.60000000000000009, 'Clay': 1.0, 'Hard': 0.90000000000000002}
{'Grass': 0.0, 'Indoor': 0.70000000000000007, 'Clay': 1.0, 'Hard': 0.90000000000000002}
{'Grass': 0.0, 'Indoor': 0.80000000000000004, 'Clay': 1.0, 'Hard': 0.90000000000000002}
{'Grass': 0.0, 'Indoor': 0.90000000000000002, 'Clay': 1.0, 'Hard': 0.90000000000000002}
{'Grass': 0.0, 'Indoor': 1.0, 'Clay': 1.0, 'Hard': 0.90000000000000002}
{'Grass': 0.0, 'Indoor': 0.0, 'Clay': 1.0, 'Hard': 1.0}
{'Grass': 0.0, 'Indoor': 0.10000000000000001, 'Clay': 1.0, 'Hard': 1.0}
{'Grass': 0.0, 'Indoor': 0.20000000000000001, 'Clay': 1.0, 'Hard': 1.0}
{'Grass': 0.0, 'Indoor': 0.30000000000000004, 'Clay': 1.0, 'Hard': 1.0}
{'Grass': 0.0, 'Indoor': 0.40000000000000002, 'Clay': 1.0, 'Hard': 1.0}
{'Grass': 0.0, 'Indoor': 0.5, 'Clay': 1.0, 'Hard': 1.0}
{'Grass': 0.0, 'Indoor': 0.60000000000000009, 'Clay': 1.0, 'Hard': 1.0}
{'Grass': 0.0, 'Indoor': 0.70000000000000007, 'Clay': 1.0, 'Hard': 1.0}
{'Grass': 0.0, 'Indoor': 0.80000

{'Grass': 0.10000000000000001, 'Indoor': 1.0, 'Clay': 1.0, 'Hard': 0.60000000000000009}
{'Grass': 0.10000000000000001, 'Indoor': 0.0, 'Clay': 1.0, 'Hard': 0.70000000000000007}
{'Grass': 0.10000000000000001, 'Indoor': 0.10000000000000001, 'Clay': 1.0, 'Hard': 0.70000000000000007}
{'Grass': 0.10000000000000001, 'Indoor': 0.20000000000000001, 'Clay': 1.0, 'Hard': 0.70000000000000007}
{'Grass': 0.10000000000000001, 'Indoor': 0.30000000000000004, 'Clay': 1.0, 'Hard': 0.70000000000000007}
{'Grass': 0.10000000000000001, 'Indoor': 0.40000000000000002, 'Clay': 1.0, 'Hard': 0.70000000000000007}
{'Grass': 0.10000000000000001, 'Indoor': 0.5, 'Clay': 1.0, 'Hard': 0.70000000000000007}
{'Grass': 0.10000000000000001, 'Indoor': 0.60000000000000009, 'Clay': 1.0, 'Hard': 0.70000000000000007}
{'Grass': 0.10000000000000001, 'Indoor': 0.70000000000000007, 'Clay': 1.0, 'Hard': 0.70000000000000007}
{'Grass': 0.10000000000000001, 'Indoor': 0.80000000000000004, 'Clay': 1.0, 'Hard': 0.70000000000000007}
{'Grass'

{'Grass': 0.20000000000000001, 'Indoor': 1.0, 'Clay': 1.0, 'Hard': 0.30000000000000004}
{'Grass': 0.20000000000000001, 'Indoor': 0.0, 'Clay': 1.0, 'Hard': 0.40000000000000002}
{'Grass': 0.20000000000000001, 'Indoor': 0.10000000000000001, 'Clay': 1.0, 'Hard': 0.40000000000000002}
{'Grass': 0.20000000000000001, 'Indoor': 0.20000000000000001, 'Clay': 1.0, 'Hard': 0.40000000000000002}
{'Grass': 0.20000000000000001, 'Indoor': 0.30000000000000004, 'Clay': 1.0, 'Hard': 0.40000000000000002}
{'Grass': 0.20000000000000001, 'Indoor': 0.40000000000000002, 'Clay': 1.0, 'Hard': 0.40000000000000002}
{'Grass': 0.20000000000000001, 'Indoor': 0.5, 'Clay': 1.0, 'Hard': 0.40000000000000002}
{'Grass': 0.20000000000000001, 'Indoor': 0.60000000000000009, 'Clay': 1.0, 'Hard': 0.40000000000000002}
{'Grass': 0.20000000000000001, 'Indoor': 0.70000000000000007, 'Clay': 1.0, 'Hard': 0.40000000000000002}
{'Grass': 0.20000000000000001, 'Indoor': 0.80000000000000004, 'Clay': 1.0, 'Hard': 0.40000000000000002}
{'Grass'

{'Grass': 0.30000000000000004, 'Indoor': 1.0, 'Clay': 1.0, 'Hard': 0.0}
{'Grass': 0.30000000000000004, 'Indoor': 0.0, 'Clay': 1.0, 'Hard': 0.10000000000000001}
{'Grass': 0.30000000000000004, 'Indoor': 0.10000000000000001, 'Clay': 1.0, 'Hard': 0.10000000000000001}
{'Grass': 0.30000000000000004, 'Indoor': 0.20000000000000001, 'Clay': 1.0, 'Hard': 0.10000000000000001}
{'Grass': 0.30000000000000004, 'Indoor': 0.30000000000000004, 'Clay': 1.0, 'Hard': 0.10000000000000001}
{'Grass': 0.30000000000000004, 'Indoor': 0.40000000000000002, 'Clay': 1.0, 'Hard': 0.10000000000000001}
{'Grass': 0.30000000000000004, 'Indoor': 0.5, 'Clay': 1.0, 'Hard': 0.10000000000000001}
{'Grass': 0.30000000000000004, 'Indoor': 0.60000000000000009, 'Clay': 1.0, 'Hard': 0.10000000000000001}
{'Grass': 0.30000000000000004, 'Indoor': 0.70000000000000007, 'Clay': 1.0, 'Hard': 0.10000000000000001}
{'Grass': 0.30000000000000004, 'Indoor': 0.80000000000000004, 'Clay': 1.0, 'Hard': 0.10000000000000001}
{'Grass': 0.300000000000

...

{'Grass': 0.70000000000000007, 'Indoor': 0.80000000000000004, 'Clay': 1.0, 'Hard': 0.60000000000000009}
{'Grass': 0.70000000000000007, 'Indoor': 0.90000000000000002, 'Clay': 1.0, 'Hard': 0.60000000000000009}
{'Grass': 0.70000000000000007, 'Indoor': 1.0, 'Clay': 1.0, 'Hard': 0.60000000000000009}
{'Grass': 0.70000000000000007, 'Indoor': 0.0, 'Clay': 1.0, 'Hard': 0.70000000000000007}
{'Grass': 0.70000000000000007, 'Indoor': 0.10000000000000001, 'Clay': 1.0, 'Hard': 0.70000000000000007}
{'Grass': 0.70000000000000007, 'Indoor': 0.20000000000000001, 'Clay': 1.0, 'Hard': 0.70000000000000007}
{'Grass': 0.70000000000000007, 'Indoor': 0.30000000000000004, 'Clay': 1.0, 'Hard': 0.70000000000000007}
{'Grass': 0.70000000000000007, 'Indoor': 0.40000000000000002, 'Clay': 1.0, 'Hard': 0.70000000000000007}
{'Grass': 0.70000000000000007, 'Indoor': 0.5, 'Clay': 1.0, 'Hard': 0.70000000000000007}
{'Grass': 0.70000000000000007, 'Indoor': 0.60000000000000009, 'Clay': 1.0, 'Hard': 0.70000000000000007}
{'Grass'

{'Grass': 0.80000000000000004, 'Indoor': 0.80000000000000004, 'Clay': 1.0, 'Hard': 0.30000000000000004}
{'Grass': 0.80000000000000004, 'Indoor': 0.90000000000000002, 'Clay': 1.0, 'Hard': 0.30000000000000004}
{'Grass': 0.80000000000000004, 'Indoor': 1.0, 'Clay': 1.0, 'Hard': 0.30000000000000004}
{'Grass': 0.80000000000000004, 'Indoor': 0.0, 'Clay': 1.0, 'Hard': 0.40000000000000002}
{'Grass': 0.80000000000000004, 'Indoor': 0.10000000000000001, 'Clay': 1.0, 'Hard': 0.40000000000000002}
{'Grass': 0.80000000000000004, 'Indoor': 0.20000000000000001, 'Clay': 1.0, 'Hard': 0.40000000000000002}
{'Grass': 0.80000000000000004, 'Indoor': 0.30000000000000004, 'Clay': 1.0, 'Hard': 0.40000000000000002}
{'Grass': 0.80000000000000004, 'Indoor': 0.40000000000000002, 'Clay': 1.0, 'Hard': 0.40000000000000002}
{'Grass': 0.80000000000000004, 'Indoor': 0.5, 'Clay': 1.0, 'Hard': 0.40000000000000002}
{'Grass': 0.80000000000000004, 'Indoor': 0.60000000000000009, 'Clay': 1.0, 'Hard': 0.40000000000000002}
{'Grass'

{'Grass': 0.90000000000000002, 'Indoor': 0.80000000000000004, 'Clay': 1.0, 'Hard': 0.0}
{'Grass': 0.90000000000000002, 'Indoor': 0.90000000000000002, 'Clay': 1.0, 'Hard': 0.0}
{'Grass': 0.90000000000000002, 'Indoor': 1.0, 'Clay': 1.0, 'Hard': 0.0}
{'Grass': 0.90000000000000002, 'Indoor': 0.0, 'Clay': 1.0, 'Hard': 0.10000000000000001}
{'Grass': 0.90000000000000002, 'Indoor': 0.10000000000000001, 'Clay': 1.0, 'Hard': 0.10000000000000001}
{'Grass': 0.90000000000000002, 'Indoor': 0.20000000000000001, 'Clay': 1.0, 'Hard': 0.10000000000000001}
{'Grass': 0.90000000000000002, 'Indoor': 0.30000000000000004, 'Clay': 1.0, 'Hard': 0.10000000000000001}
{'Grass': 0.90000000000000002, 'Indoor': 0.40000000000000002, 'Clay': 1.0, 'Hard': 0.10000000000000001}
{'Grass': 0.90000000000000002, 'Indoor': 0.5, 'Clay': 1.0, 'Hard': 0.10000000000000001}
{'Grass': 0.90000000000000002, 'Indoor': 0.60000000000000009, 'Clay': 1.0, 'Hard': 0.10000000000000001}
{'Grass': 0.90000000000000002, 'Indoor': 0.7000000000000

{'Grass': 0.90000000000000002, 'Indoor': 0.80000000000000004, 'Clay': 1.0, 'Hard': 0.80000000000000004}
{'Grass': 0.90000000000000002, 'Indoor': 0.90000000000000002, 'Clay': 1.0, 'Hard': 0.80000000000000004}
{'Grass': 0.90000000000000002, 'Indoor': 1.0, 'Clay': 1.0, 'Hard': 0.80000000000000004}
{'Grass': 0.90000000000000002, 'Indoor': 0.0, 'Clay': 1.0, 'Hard': 0.90000000000000002}
{'Grass': 0.90000000000000002, 'Indoor': 0.10000000000000001, 'Clay': 1.0, 'Hard': 0.90000000000000002}
{'Grass': 0.90000000000000002, 'Indoor': 0.20000000000000001, 'Clay': 1.0, 'Hard': 0.90000000000000002}
{'Grass': 0.90000000000000002, 'Indoor': 0.30000000000000004, 'Clay': 1.0, 'Hard': 0.90000000000000002}
{'Grass': 0.90000000000000002, 'Indoor': 0.40000000000000002, 'Clay': 1.0, 'Hard': 0.90000000000000002}
{'Grass': 0.90000000000000002, 'Indoor': 0.5, 'Clay': 1.0, 'Hard': 0.90000000000000002}
{'Grass': 0.90000000000000002, 'Indoor': 0.60000000000000009, 'Clay': 1.0, 'Hard': 0.90000000000000002}
{'Grass'

{'Grass': 1.0, 'Indoor': 0.90000000000000002, 'Clay': 1.0, 'Hard': 0.60000000000000009}
{'Grass': 1.0, 'Indoor': 1.0, 'Clay': 1.0, 'Hard': 0.60000000000000009}
{'Grass': 1.0, 'Indoor': 0.0, 'Clay': 1.0, 'Hard': 0.70000000000000007}
{'Grass': 1.0, 'Indoor': 0.10000000000000001, 'Clay': 1.0, 'Hard': 0.70000000000000007}
{'Grass': 1.0, 'Indoor': 0.20000000000000001, 'Clay': 1.0, 'Hard': 0.70000000000000007}
{'Grass': 1.0, 'Indoor': 0.30000000000000004, 'Clay': 1.0, 'Hard': 0.70000000000000007}
{'Grass': 1.0, 'Indoor': 0.40000000000000002, 'Clay': 1.0, 'Hard': 0.70000000000000007}
{'Grass': 1.0, 'Indoor': 0.5, 'Clay': 1.0, 'Hard': 0.70000000000000007}
{'Grass': 1.0, 'Indoor': 0.60000000000000009, 'Clay': 1.0, 'Hard': 0.70000000000000007}
{'Grass': 1.0, 'Indoor': 0.70000000000000007, 'Clay': 1.0, 'Hard': 0.70000000000000007}
{'Grass': 1.0, 'Indoor': 0.80000000000000004, 'Clay': 1.0, 'Hard': 0.70000000000000007}
{'Grass': 1.0, 'Indoor': 0.90000000000000002, 'Clay': 1.0, 'Hard': 0.70000000000

In [10]:
clay_results.sort('auc', ascending=False).head()

/Users/siddhantjagadish/Documents/DataProjects/tennis_modeling/tennis_modeling/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  """Entry point for launching an IPython kernel.


,Grass,Hard,Indoor,auc,accuracy
366,0.3,0.0,0.3,0.728363,0.670471
245,0.2,0.0,0.3,0.728281,0.669044
244,0.2,0.0,0.2,0.728265,0.666191
255,0.2,0.1,0.2,0.728183,0.673324
123,0.1,0.0,0.2,0.728142,0.664765


In [11]:
grass_results = tune_weights_for_surface('Grass')

{'Grass': 1.0, 'Hard': 0.0, 'Indoor': 0.0, 'Clay': 0.0}
{'Grass': 1.0, 'Hard': 0.0, 'Indoor': 0.10000000000000001, 'Clay': 0.0}
{'Grass': 1.0, 'Hard': 0.0, 'Indoor': 0.20000000000000001, 'Clay': 0.0}
{'Grass': 1.0, 'Hard': 0.0, 'Indoor': 0.30000000000000004, 'Clay': 0.0}
{'Grass': 1.0, 'Hard': 0.0, 'Indoor': 0.40000000000000002, 'Clay': 0.0}
{'Grass': 1.0, 'Hard': 0.0, 'Indoor': 0.5, 'Clay': 0.0}
{'Grass': 1.0, 'Hard': 0.0, 'Indoor': 0.60000000000000009, 'Clay': 0.0}
{'Grass': 1.0, 'Hard': 0.0, 'Indoor': 0.70000000000000007, 'Clay': 0.0}
{'Grass': 1.0, 'Hard': 0.0, 'Indoor': 0.80000000000000004, 'Clay': 0.0}
{'Grass': 1.0, 'Hard': 0.0, 'Indoor': 0.90000000000000002, 'Clay': 0.0}
{'Grass': 1.0, 'Hard': 0.0, 'Indoor': 1.0, 'Clay': 0.0}
{'Grass': 1.0, 'Hard': 0.10000000000000001, 'Indoor': 0.0, 'Clay': 0.0}
{'Grass': 1.0, 'Hard': 0.10000000000000001, 'Indoor': 0.10000000000000001, 'Clay': 0.0}
{'Grass': 1.0, 'Hard': 0.10000000000000001, 'Indoor': 0.20000000000000001, 'Clay': 0.0}
{'Grass'

{'Grass': 1.0, 'Hard': 0.90000000000000002, 'Indoor': 0.60000000000000009, 'Clay': 0.0}
{'Grass': 1.0, 'Hard': 0.90000000000000002, 'Indoor': 0.70000000000000007, 'Clay': 0.0}
{'Grass': 1.0, 'Hard': 0.90000000000000002, 'Indoor': 0.80000000000000004, 'Clay': 0.0}
{'Grass': 1.0, 'Hard': 0.90000000000000002, 'Indoor': 0.90000000000000002, 'Clay': 0.0}
{'Grass': 1.0, 'Hard': 0.90000000000000002, 'Indoor': 1.0, 'Clay': 0.0}
{'Grass': 1.0, 'Hard': 1.0, 'Indoor': 0.0, 'Clay': 0.0}
{'Grass': 1.0, 'Hard': 1.0, 'Indoor': 0.10000000000000001, 'Clay': 0.0}
{'Grass': 1.0, 'Hard': 1.0, 'Indoor': 0.20000000000000001, 'Clay': 0.0}
{'Grass': 1.0, 'Hard': 1.0, 'Indoor': 0.30000000000000004, 'Clay': 0.0}
{'Grass': 1.0, 'Hard': 1.0, 'Indoor': 0.40000000000000002, 'Clay': 0.0}
{'Grass': 1.0, 'Hard': 1.0, 'Indoor': 0.5, 'Clay': 0.0}
{'Grass': 1.0, 'Hard': 1.0, 'Indoor': 0.60000000000000009, 'Clay': 0.0}
{'Grass': 1.0, 'Hard': 1.0, 'Indoor': 0.70000000000000007, 'Clay': 0.0}
{'Grass': 1.0, 'Hard': 1.0, 'Ind

{'Grass': 1.0, 'Hard': 0.60000000000000009, 'Indoor': 0.90000000000000002, 'Clay': 0.10000000000000001}
{'Grass': 1.0, 'Hard': 0.60000000000000009, 'Indoor': 1.0, 'Clay': 0.10000000000000001}
{'Grass': 1.0, 'Hard': 0.70000000000000007, 'Indoor': 0.0, 'Clay': 0.10000000000000001}
{'Grass': 1.0, 'Hard': 0.70000000000000007, 'Indoor': 0.10000000000000001, 'Clay': 0.10000000000000001}
{'Grass': 1.0, 'Hard': 0.70000000000000007, 'Indoor': 0.20000000000000001, 'Clay': 0.10000000000000001}
{'Grass': 1.0, 'Hard': 0.70000000000000007, 'Indoor': 0.30000000000000004, 'Clay': 0.10000000000000001}
{'Grass': 1.0, 'Hard': 0.70000000000000007, 'Indoor': 0.40000000000000002, 'Clay': 0.10000000000000001}
{'Grass': 1.0, 'Hard': 0.70000000000000007, 'Indoor': 0.5, 'Clay': 0.10000000000000001}
{'Grass': 1.0, 'Hard': 0.70000000000000007, 'Indoor': 0.60000000000000009, 'Clay': 0.10000000000000001}
{'Grass': 1.0, 'Hard': 0.70000000000000007, 'Indoor': 0.70000000000000007, 'Clay': 0.10000000000000001}
{'Grass'

{'Grass': 1.0, 'Hard': 0.30000000000000004, 'Indoor': 0.80000000000000004, 'Clay': 0.20000000000000001}
{'Grass': 1.0, 'Hard': 0.30000000000000004, 'Indoor': 0.90000000000000002, 'Clay': 0.20000000000000001}
{'Grass': 1.0, 'Hard': 0.30000000000000004, 'Indoor': 1.0, 'Clay': 0.20000000000000001}
{'Grass': 1.0, 'Hard': 0.40000000000000002, 'Indoor': 0.0, 'Clay': 0.20000000000000001}
{'Grass': 1.0, 'Hard': 0.40000000000000002, 'Indoor': 0.10000000000000001, 'Clay': 0.20000000000000001}
{'Grass': 1.0, 'Hard': 0.40000000000000002, 'Indoor': 0.20000000000000001, 'Clay': 0.20000000000000001}
{'Grass': 1.0, 'Hard': 0.40000000000000002, 'Indoor': 0.30000000000000004, 'Clay': 0.20000000000000001}
{'Grass': 1.0, 'Hard': 0.40000000000000002, 'Indoor': 0.40000000000000002, 'Clay': 0.20000000000000001}
{'Grass': 1.0, 'Hard': 0.40000000000000002, 'Indoor': 0.5, 'Clay': 0.20000000000000001}
{'Grass': 1.0, 'Hard': 0.40000000000000002, 'Indoor': 0.60000000000000009, 'Clay': 0.20000000000000001}
{'Grass'

{'Grass': 1.0, 'Hard': 0.0, 'Indoor': 0.80000000000000004, 'Clay': 0.30000000000000004}
{'Grass': 1.0, 'Hard': 0.0, 'Indoor': 0.90000000000000002, 'Clay': 0.30000000000000004}
{'Grass': 1.0, 'Hard': 0.0, 'Indoor': 1.0, 'Clay': 0.30000000000000004}
{'Grass': 1.0, 'Hard': 0.10000000000000001, 'Indoor': 0.0, 'Clay': 0.30000000000000004}
{'Grass': 1.0, 'Hard': 0.10000000000000001, 'Indoor': 0.10000000000000001, 'Clay': 0.30000000000000004}
{'Grass': 1.0, 'Hard': 0.10000000000000001, 'Indoor': 0.20000000000000001, 'Clay': 0.30000000000000004}
{'Grass': 1.0, 'Hard': 0.10000000000000001, 'Indoor': 0.30000000000000004, 'Clay': 0.30000000000000004}
{'Grass': 1.0, 'Hard': 0.10000000000000001, 'Indoor': 0.40000000000000002, 'Clay': 0.30000000000000004}
{'Grass': 1.0, 'Hard': 0.10000000000000001, 'Indoor': 0.5, 'Clay': 0.30000000000000004}
{'Grass': 1.0, 'Hard': 0.10000000000000001, 'Indoor': 0.60000000000000009, 'Clay': 0.30000000000000004}
{'Grass': 1.0, 'Hard': 0.10000000000000001, 'Indoor': 0.

...

{'Grass': 1.0, 'Hard': 0.70000000000000007, 'Indoor': 0.10000000000000001, 'Clay': 0.70000000000000007}
{'Grass': 1.0, 'Hard': 0.70000000000000007, 'Indoor': 0.20000000000000001, 'Clay': 0.70000000000000007}
{'Grass': 1.0, 'Hard': 0.70000000000000007, 'Indoor': 0.30000000000000004, 'Clay': 0.70000000000000007}
{'Grass': 1.0, 'Hard': 0.70000000000000007, 'Indoor': 0.40000000000000002, 'Clay': 0.70000000000000007}
{'Grass': 1.0, 'Hard': 0.70000000000000007, 'Indoor': 0.5, 'Clay': 0.70000000000000007}
{'Grass': 1.0, 'Hard': 0.70000000000000007, 'Indoor': 0.60000000000000009, 'Clay': 0.70000000000000007}
{'Grass': 1.0, 'Hard': 0.70000000000000007, 'Indoor': 0.70000000000000007, 'Clay': 0.70000000000000007}
{'Grass': 1.0, 'Hard': 0.70000000000000007, 'Indoor': 0.80000000000000004, 'Clay': 0.70000000000000007}
{'Grass': 1.0, 'Hard': 0.70000000000000007, 'Indoor': 0.90000000000000002, 'Clay': 0.70000000000000007}
{'Grass': 1.0, 'Hard': 0.70000000000000007, 'Indoor': 1.0, 'Clay': 0.70000000000

{'Grass': 1.0, 'Hard': 0.30000000000000004, 'Indoor': 1.0, 'Clay': 0.80000000000000004}
{'Grass': 1.0, 'Hard': 0.40000000000000002, 'Indoor': 0.0, 'Clay': 0.80000000000000004}
{'Grass': 1.0, 'Hard': 0.40000000000000002, 'Indoor': 0.10000000000000001, 'Clay': 0.80000000000000004}
{'Grass': 1.0, 'Hard': 0.40000000000000002, 'Indoor': 0.20000000000000001, 'Clay': 0.80000000000000004}
{'Grass': 1.0, 'Hard': 0.40000000000000002, 'Indoor': 0.30000000000000004, 'Clay': 0.80000000000000004}
{'Grass': 1.0, 'Hard': 0.40000000000000002, 'Indoor': 0.40000000000000002, 'Clay': 0.80000000000000004}
{'Grass': 1.0, 'Hard': 0.40000000000000002, 'Indoor': 0.5, 'Clay': 0.80000000000000004}
{'Grass': 1.0, 'Hard': 0.40000000000000002, 'Indoor': 0.60000000000000009, 'Clay': 0.80000000000000004}
{'Grass': 1.0, 'Hard': 0.40000000000000002, 'Indoor': 0.70000000000000007, 'Clay': 0.80000000000000004}
{'Grass': 1.0, 'Hard': 0.40000000000000002, 'Indoor': 0.80000000000000004, 'Clay': 0.80000000000000004}
{'Grass'

{'Grass': 1.0, 'Hard': 0.0, 'Indoor': 1.0, 'Clay': 0.90000000000000002}
{'Grass': 1.0, 'Hard': 0.10000000000000001, 'Indoor': 0.0, 'Clay': 0.90000000000000002}
{'Grass': 1.0, 'Hard': 0.10000000000000001, 'Indoor': 0.10000000000000001, 'Clay': 0.90000000000000002}
{'Grass': 1.0, 'Hard': 0.10000000000000001, 'Indoor': 0.20000000000000001, 'Clay': 0.90000000000000002}
{'Grass': 1.0, 'Hard': 0.10000000000000001, 'Indoor': 0.30000000000000004, 'Clay': 0.90000000000000002}
{'Grass': 1.0, 'Hard': 0.10000000000000001, 'Indoor': 0.40000000000000002, 'Clay': 0.90000000000000002}
{'Grass': 1.0, 'Hard': 0.10000000000000001, 'Indoor': 0.5, 'Clay': 0.90000000000000002}
{'Grass': 1.0, 'Hard': 0.10000000000000001, 'Indoor': 0.60000000000000009, 'Clay': 0.90000000000000002}
{'Grass': 1.0, 'Hard': 0.10000000000000001, 'Indoor': 0.70000000000000007, 'Clay': 0.90000000000000002}
{'Grass': 1.0, 'Hard': 0.10000000000000001, 'Indoor': 0.80000000000000004, 'Clay': 0.90000000000000002}
{'Grass': 1.0, 'Hard': 0

{'Grass': 1.0, 'Hard': 0.80000000000000004, 'Indoor': 1.0, 'Clay': 0.90000000000000002}
{'Grass': 1.0, 'Hard': 0.90000000000000002, 'Indoor': 0.0, 'Clay': 0.90000000000000002}
{'Grass': 1.0, 'Hard': 0.90000000000000002, 'Indoor': 0.10000000000000001, 'Clay': 0.90000000000000002}
{'Grass': 1.0, 'Hard': 0.90000000000000002, 'Indoor': 0.20000000000000001, 'Clay': 0.90000000000000002}
{'Grass': 1.0, 'Hard': 0.90000000000000002, 'Indoor': 0.30000000000000004, 'Clay': 0.90000000000000002}
{'Grass': 1.0, 'Hard': 0.90000000000000002, 'Indoor': 0.40000000000000002, 'Clay': 0.90000000000000002}
{'Grass': 1.0, 'Hard': 0.90000000000000002, 'Indoor': 0.5, 'Clay': 0.90000000000000002}
{'Grass': 1.0, 'Hard': 0.90000000000000002, 'Indoor': 0.60000000000000009, 'Clay': 0.90000000000000002}
{'Grass': 1.0, 'Hard': 0.90000000000000002, 'Indoor': 0.70000000000000007, 'Clay': 0.90000000000000002}
{'Grass': 1.0, 'Hard': 0.90000000000000002, 'Indoor': 0.80000000000000004, 'Clay': 0.90000000000000002}
{'Grass'

{'Grass': 1.0, 'Hard': 0.70000000000000007, 'Indoor': 0.40000000000000002, 'Clay': 1.0}
{'Grass': 1.0, 'Hard': 0.70000000000000007, 'Indoor': 0.5, 'Clay': 1.0}
{'Grass': 1.0, 'Hard': 0.70000000000000007, 'Indoor': 0.60000000000000009, 'Clay': 1.0}
{'Grass': 1.0, 'Hard': 0.70000000000000007, 'Indoor': 0.70000000000000007, 'Clay': 1.0}
{'Grass': 1.0, 'Hard': 0.70000000000000007, 'Indoor': 0.80000000000000004, 'Clay': 1.0}
{'Grass': 1.0, 'Hard': 0.70000000000000007, 'Indoor': 0.90000000000000002, 'Clay': 1.0}
{'Grass': 1.0, 'Hard': 0.70000000000000007, 'Indoor': 1.0, 'Clay': 1.0}
{'Grass': 1.0, 'Hard': 0.80000000000000004, 'Indoor': 0.0, 'Clay': 1.0}
{'Grass': 1.0, 'Hard': 0.80000000000000004, 'Indoor': 0.10000000000000001, 'Clay': 1.0}
{'Grass': 1.0, 'Hard': 0.80000000000000004, 'Indoor': 0.20000000000000001, 'Clay': 1.0}
{'Grass': 1.0, 'Hard': 0.80000000000000004, 'Indoor': 0.30000000000000004, 'Clay': 1.0}
{'Grass': 1.0, 'Hard': 0.80000000000000004, 'Indoor': 0.40000000000000002, 'Clay

In [12]:
grass_results.sort('auc', ascending=False).head()

/Users/siddhantjagadish/Documents/DataProjects/tennis_modeling/tennis_modeling/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  """Entry point for launching an IPython kernel.


,Clay,Hard,Indoor,auc,accuracy
352,0.2,1.0,0.0,0.734306,0.684211
353,0.2,1.0,0.1,0.734306,0.687719
233,0.1,1.0,0.2,0.734158,0.687719
231,0.1,1.0,0.0,0.733912,0.684211
232,0.1,1.0,0.1,0.733912,0.687719


In [13]:
indoor_results = tune_weights_for_surface('Indoor')

{'Hard': 0.0, 'Grass': 0.0, 'Indoor': 1.0, 'Clay': 0.0}
{'Hard': 0.10000000000000001, 'Grass': 0.0, 'Indoor': 1.0, 'Clay': 0.0}
{'Hard': 0.20000000000000001, 'Grass': 0.0, 'Indoor': 1.0, 'Clay': 0.0}
{'Hard': 0.30000000000000004, 'Grass': 0.0, 'Indoor': 1.0, 'Clay': 0.0}
{'Hard': 0.40000000000000002, 'Grass': 0.0, 'Indoor': 1.0, 'Clay': 0.0}
{'Hard': 0.5, 'Grass': 0.0, 'Indoor': 1.0, 'Clay': 0.0}
{'Hard': 0.60000000000000009, 'Grass': 0.0, 'Indoor': 1.0, 'Clay': 0.0}
{'Hard': 0.70000000000000007, 'Grass': 0.0, 'Indoor': 1.0, 'Clay': 0.0}
{'Hard': 0.80000000000000004, 'Grass': 0.0, 'Indoor': 1.0, 'Clay': 0.0}
{'Hard': 0.90000000000000002, 'Grass': 0.0, 'Indoor': 1.0, 'Clay': 0.0}
{'Hard': 1.0, 'Grass': 0.0, 'Indoor': 1.0, 'Clay': 0.0}
{'Hard': 0.0, 'Grass': 0.10000000000000001, 'Indoor': 1.0, 'Clay': 0.0}
{'Hard': 0.10000000000000001, 'Grass': 0.10000000000000001, 'Indoor': 1.0, 'Clay': 0.0}
{'Hard': 0.20000000000000001, 'Grass': 0.10000000000000001, 'Indoor': 1.0, 'Clay': 0.0}
{'Hard':

{'Hard': 0.70000000000000007, 'Grass': 0.90000000000000002, 'Indoor': 1.0, 'Clay': 0.0}
{'Hard': 0.80000000000000004, 'Grass': 0.90000000000000002, 'Indoor': 1.0, 'Clay': 0.0}
{'Hard': 0.90000000000000002, 'Grass': 0.90000000000000002, 'Indoor': 1.0, 'Clay': 0.0}
{'Hard': 1.0, 'Grass': 0.90000000000000002, 'Indoor': 1.0, 'Clay': 0.0}
{'Hard': 0.0, 'Grass': 1.0, 'Indoor': 1.0, 'Clay': 0.0}
{'Hard': 0.10000000000000001, 'Grass': 1.0, 'Indoor': 1.0, 'Clay': 0.0}
{'Hard': 0.20000000000000001, 'Grass': 1.0, 'Indoor': 1.0, 'Clay': 0.0}
{'Hard': 0.30000000000000004, 'Grass': 1.0, 'Indoor': 1.0, 'Clay': 0.0}
{'Hard': 0.40000000000000002, 'Grass': 1.0, 'Indoor': 1.0, 'Clay': 0.0}
{'Hard': 0.5, 'Grass': 1.0, 'Indoor': 1.0, 'Clay': 0.0}
{'Hard': 0.60000000000000009, 'Grass': 1.0, 'Indoor': 1.0, 'Clay': 0.0}
{'Hard': 0.70000000000000007, 'Grass': 1.0, 'Indoor': 1.0, 'Clay': 0.0}
{'Hard': 0.80000000000000004, 'Grass': 1.0, 'Indoor': 1.0, 'Clay': 0.0}
{'Hard': 0.90000000000000002, 'Grass': 1.0, 'Ind

{'Hard': 0.0, 'Grass': 0.70000000000000007, 'Indoor': 1.0, 'Clay': 0.10000000000000001}
{'Hard': 0.10000000000000001, 'Grass': 0.70000000000000007, 'Indoor': 1.0, 'Clay': 0.10000000000000001}
{'Hard': 0.20000000000000001, 'Grass': 0.70000000000000007, 'Indoor': 1.0, 'Clay': 0.10000000000000001}
{'Hard': 0.30000000000000004, 'Grass': 0.70000000000000007, 'Indoor': 1.0, 'Clay': 0.10000000000000001}
{'Hard': 0.40000000000000002, 'Grass': 0.70000000000000007, 'Indoor': 1.0, 'Clay': 0.10000000000000001}
{'Hard': 0.5, 'Grass': 0.70000000000000007, 'Indoor': 1.0, 'Clay': 0.10000000000000001}
{'Hard': 0.60000000000000009, 'Grass': 0.70000000000000007, 'Indoor': 1.0, 'Clay': 0.10000000000000001}
{'Hard': 0.70000000000000007, 'Grass': 0.70000000000000007, 'Indoor': 1.0, 'Clay': 0.10000000000000001}
{'Hard': 0.80000000000000004, 'Grass': 0.70000000000000007, 'Indoor': 1.0, 'Clay': 0.10000000000000001}
{'Hard': 0.90000000000000002, 'Grass': 0.70000000000000007, 'Indoor': 1.0, 'Clay': 0.10000000000

{'Hard': 0.10000000000000001, 'Grass': 0.40000000000000002, 'Indoor': 1.0, 'Clay': 0.20000000000000001}
{'Hard': 0.20000000000000001, 'Grass': 0.40000000000000002, 'Indoor': 1.0, 'Clay': 0.20000000000000001}
{'Hard': 0.30000000000000004, 'Grass': 0.40000000000000002, 'Indoor': 1.0, 'Clay': 0.20000000000000001}
{'Hard': 0.40000000000000002, 'Grass': 0.40000000000000002, 'Indoor': 1.0, 'Clay': 0.20000000000000001}
{'Hard': 0.5, 'Grass': 0.40000000000000002, 'Indoor': 1.0, 'Clay': 0.20000000000000001}
{'Hard': 0.60000000000000009, 'Grass': 0.40000000000000002, 'Indoor': 1.0, 'Clay': 0.20000000000000001}
{'Hard': 0.70000000000000007, 'Grass': 0.40000000000000002, 'Indoor': 1.0, 'Clay': 0.20000000000000001}
{'Hard': 0.80000000000000004, 'Grass': 0.40000000000000002, 'Indoor': 1.0, 'Clay': 0.20000000000000001}
{'Hard': 0.90000000000000002, 'Grass': 0.40000000000000002, 'Indoor': 1.0, 'Clay': 0.20000000000000001}
{'Hard': 1.0, 'Grass': 0.40000000000000002, 'Indoor': 1.0, 'Clay': 0.20000000000

{'Hard': 0.10000000000000001, 'Grass': 0.10000000000000001, 'Indoor': 1.0, 'Clay': 0.30000000000000004}
{'Hard': 0.20000000000000001, 'Grass': 0.10000000000000001, 'Indoor': 1.0, 'Clay': 0.30000000000000004}
{'Hard': 0.30000000000000004, 'Grass': 0.10000000000000001, 'Indoor': 1.0, 'Clay': 0.30000000000000004}
{'Hard': 0.40000000000000002, 'Grass': 0.10000000000000001, 'Indoor': 1.0, 'Clay': 0.30000000000000004}
{'Hard': 0.5, 'Grass': 0.10000000000000001, 'Indoor': 1.0, 'Clay': 0.30000000000000004}
{'Hard': 0.60000000000000009, 'Grass': 0.10000000000000001, 'Indoor': 1.0, 'Clay': 0.30000000000000004}
{'Hard': 0.70000000000000007, 'Grass': 0.10000000000000001, 'Indoor': 1.0, 'Clay': 0.30000000000000004}
{'Hard': 0.80000000000000004, 'Grass': 0.10000000000000001, 'Indoor': 1.0, 'Clay': 0.30000000000000004}
{'Hard': 0.90000000000000002, 'Grass': 0.10000000000000001, 'Indoor': 1.0, 'Clay': 0.30000000000000004}
{'Hard': 1.0, 'Grass': 0.10000000000000001, 'Indoor': 1.0, 'Clay': 0.30000000000

...

{'Hard': 0.60000000000000009, 'Grass': 0.60000000000000009, 'Indoor': 1.0, 'Clay': 0.70000000000000007}
{'Hard': 0.70000000000000007, 'Grass': 0.60000000000000009, 'Indoor': 1.0, 'Clay': 0.70000000000000007}
{'Hard': 0.80000000000000004, 'Grass': 0.60000000000000009, 'Indoor': 1.0, 'Clay': 0.70000000000000007}
{'Hard': 0.90000000000000002, 'Grass': 0.60000000000000009, 'Indoor': 1.0, 'Clay': 0.70000000000000007}
{'Hard': 1.0, 'Grass': 0.60000000000000009, 'Indoor': 1.0, 'Clay': 0.70000000000000007}
{'Hard': 0.0, 'Grass': 0.70000000000000007, 'Indoor': 1.0, 'Clay': 0.70000000000000007}
{'Hard': 0.10000000000000001, 'Grass': 0.70000000000000007, 'Indoor': 1.0, 'Clay': 0.70000000000000007}
{'Hard': 0.20000000000000001, 'Grass': 0.70000000000000007, 'Indoor': 1.0, 'Clay': 0.70000000000000007}
{'Hard': 0.30000000000000004, 'Grass': 0.70000000000000007, 'Indoor': 1.0, 'Clay': 0.70000000000000007}
{'Hard': 0.40000000000000002, 'Grass': 0.70000000000000007, 'Indoor': 1.0, 'Clay': 0.70000000000

{'Hard': 0.60000000000000009, 'Grass': 0.30000000000000004, 'Indoor': 1.0, 'Clay': 0.80000000000000004}
{'Hard': 0.70000000000000007, 'Grass': 0.30000000000000004, 'Indoor': 1.0, 'Clay': 0.80000000000000004}
{'Hard': 0.80000000000000004, 'Grass': 0.30000000000000004, 'Indoor': 1.0, 'Clay': 0.80000000000000004}
{'Hard': 0.90000000000000002, 'Grass': 0.30000000000000004, 'Indoor': 1.0, 'Clay': 0.80000000000000004}
{'Hard': 1.0, 'Grass': 0.30000000000000004, 'Indoor': 1.0, 'Clay': 0.80000000000000004}
{'Hard': 0.0, 'Grass': 0.40000000000000002, 'Indoor': 1.0, 'Clay': 0.80000000000000004}
{'Hard': 0.10000000000000001, 'Grass': 0.40000000000000002, 'Indoor': 1.0, 'Clay': 0.80000000000000004}
{'Hard': 0.20000000000000001, 'Grass': 0.40000000000000002, 'Indoor': 1.0, 'Clay': 0.80000000000000004}
{'Hard': 0.30000000000000004, 'Grass': 0.40000000000000002, 'Indoor': 1.0, 'Clay': 0.80000000000000004}
{'Hard': 0.40000000000000002, 'Grass': 0.40000000000000002, 'Indoor': 1.0, 'Clay': 0.80000000000

{'Hard': 0.60000000000000009, 'Grass': 0.0, 'Indoor': 1.0, 'Clay': 0.90000000000000002}
{'Hard': 0.70000000000000007, 'Grass': 0.0, 'Indoor': 1.0, 'Clay': 0.90000000000000002}
{'Hard': 0.80000000000000004, 'Grass': 0.0, 'Indoor': 1.0, 'Clay': 0.90000000000000002}
{'Hard': 0.90000000000000002, 'Grass': 0.0, 'Indoor': 1.0, 'Clay': 0.90000000000000002}
{'Hard': 1.0, 'Grass': 0.0, 'Indoor': 1.0, 'Clay': 0.90000000000000002}
{'Hard': 0.0, 'Grass': 0.10000000000000001, 'Indoor': 1.0, 'Clay': 0.90000000000000002}
{'Hard': 0.10000000000000001, 'Grass': 0.10000000000000001, 'Indoor': 1.0, 'Clay': 0.90000000000000002}
{'Hard': 0.20000000000000001, 'Grass': 0.10000000000000001, 'Indoor': 1.0, 'Clay': 0.90000000000000002}
{'Hard': 0.30000000000000004, 'Grass': 0.10000000000000001, 'Indoor': 1.0, 'Clay': 0.90000000000000002}
{'Hard': 0.40000000000000002, 'Grass': 0.10000000000000001, 'Indoor': 1.0, 'Clay': 0.90000000000000002}
{'Hard': 0.5, 'Grass': 0.10000000000000001, 'Indoor': 1.0, 'Clay': 0.900

{'Hard': 0.60000000000000009, 'Grass': 0.80000000000000004, 'Indoor': 1.0, 'Clay': 0.90000000000000002}
{'Hard': 0.70000000000000007, 'Grass': 0.80000000000000004, 'Indoor': 1.0, 'Clay': 0.90000000000000002}
{'Hard': 0.80000000000000004, 'Grass': 0.80000000000000004, 'Indoor': 1.0, 'Clay': 0.90000000000000002}
{'Hard': 0.90000000000000002, 'Grass': 0.80000000000000004, 'Indoor': 1.0, 'Clay': 0.90000000000000002}
{'Hard': 1.0, 'Grass': 0.80000000000000004, 'Indoor': 1.0, 'Clay': 0.90000000000000002}
{'Hard': 0.0, 'Grass': 0.90000000000000002, 'Indoor': 1.0, 'Clay': 0.90000000000000002}
{'Hard': 0.10000000000000001, 'Grass': 0.90000000000000002, 'Indoor': 1.0, 'Clay': 0.90000000000000002}
{'Hard': 0.20000000000000001, 'Grass': 0.90000000000000002, 'Indoor': 1.0, 'Clay': 0.90000000000000002}
{'Hard': 0.30000000000000004, 'Grass': 0.90000000000000002, 'Indoor': 1.0, 'Clay': 0.90000000000000002}
{'Hard': 0.40000000000000002, 'Grass': 0.90000000000000002, 'Indoor': 1.0, 'Clay': 0.90000000000

{'Hard': 0.70000000000000007, 'Grass': 0.60000000000000009, 'Indoor': 1.0, 'Clay': 1.0}
{'Hard': 0.80000000000000004, 'Grass': 0.60000000000000009, 'Indoor': 1.0, 'Clay': 1.0}
{'Hard': 0.90000000000000002, 'Grass': 0.60000000000000009, 'Indoor': 1.0, 'Clay': 1.0}
{'Hard': 1.0, 'Grass': 0.60000000000000009, 'Indoor': 1.0, 'Clay': 1.0}
{'Hard': 0.0, 'Grass': 0.70000000000000007, 'Indoor': 1.0, 'Clay': 1.0}
{'Hard': 0.10000000000000001, 'Grass': 0.70000000000000007, 'Indoor': 1.0, 'Clay': 1.0}
{'Hard': 0.20000000000000001, 'Grass': 0.70000000000000007, 'Indoor': 1.0, 'Clay': 1.0}
{'Hard': 0.30000000000000004, 'Grass': 0.70000000000000007, 'Indoor': 1.0, 'Clay': 1.0}
{'Hard': 0.40000000000000002, 'Grass': 0.70000000000000007, 'Indoor': 1.0, 'Clay': 1.0}
{'Hard': 0.5, 'Grass': 0.70000000000000007, 'Indoor': 1.0, 'Clay': 1.0}
{'Hard': 0.60000000000000009, 'Grass': 0.70000000000000007, 'Indoor': 1.0, 'Clay': 1.0}
{'Hard': 0.70000000000000007, 'Grass': 0.70000000000000007, 'Indoor': 1.0, 'Clay

In [14]:
indoor_results.sort('auc', ascending=False).head()

/Users/siddhantjagadish/Documents/DataProjects/tennis_modeling/tennis_modeling/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  """Entry point for launching an IPython kernel.


,Clay,Grass,Hard,auc,accuracy
1319,1.0,0.9,1.0,0.683634,0.650538
1326,1.0,1.0,0.6,0.683634,0.645161
1330,1.0,1.0,1.0,0.683518,0.653226
1328,1.0,1.0,0.8,0.683518,0.650538
1327,1.0,1.0,0.7,0.683345,0.650538


Why is indoor performance so bad???  Are there strange tournaments here?  Is it hard vs. carpet vs. clay?  We'll come back to this...

In [15]:
df[df['court'] == 'Indoor']['surface'].value_counts()

Hard      6559
Carpet    1587
Clay       170
Name: surface, dtype: int64

There are a lot of carpet matches...should we throw these out?

#### Evaluate Betting Performance 

In [ ]:
def evaluate_betting(val_df, buff=0):
    val_df['bet1'] = (1. / val_df['p1_odds']) < (val_df['cal_pred'] - buff)
    val_df['bet2'] = (1. / val_df['p2_odds']) < (1. - val_df['cal_pred'] - buff)
    bet_revenues = (
        val_df['p1_odds'] * val_df['bet1'] * val_df['y']  +
        val_df['p2_odds'] * val_df['bet2'] * (1. - val_df['y'])
    )
    bet_spending = val_df['bet1'] + val_df['bet2']
    profit_over_time = bet_revenues.cumsum() - bet_spending.cumsum()
    
    total_placed = bet_spending.sum()
    total_won = (val_df['bet1'] * val_df['y']).sum() + (val_df['bet2'] * (1. - val_df['y'])).sum()
    return bet_revenues.sum() - bet_spending.sum(), profit_over_time 

In [ ]:
units_won, over_time = evaluate_betting(val_df)
plt.plot(over_time)

In [ ]:
units_won

#### Compare to always betting on player 1

Let's make sure we lose money if we just choose a random player

In [ ]:
new_val_df = val_df.copy()
new_val_df['cal_pred'] = 0.

units_won, over_time = evaluate_betting(new_val_df)
plt.plot(over_time)

Cool, we lose a ton of money by randomly choosing a player

In [ ]:
import numpy as np

val_df['tot_probs'] = ((1 / val_df['p1_odds']) + (1. / val_df['p2_odds']))

In [ ]:
val_df[['p1_odds', 'p2_odds', 'winner', 'loser', 'maxw', 'maxl', 'tot_probs']][val_df['tot_probs'] < 1.]